In [ ]:
import numpy as np
import niche
import ATN
import multiprocessing
from multiprocessing import Pool
from multiprocessing import Array
from multiprocessing import Value

# CONSTANTS
out_location = "paper_webs"
dummy_location = "NONE" # Don't re-run random web generation

In [4]:
### Generate initial webs (random). ###
### To use the same webs as in the paper don't re-run and use those in paper_webs ###

S = 30 
C = 0.15

# Generate 100 potential webs using the niche model, saving a set of plausible ones
webs0 = []
for i in range(0,100):
    nis, ris, cis, web = niche.niche_model(S,C)
    b_init = {}
    for node in web.nodes():
        # set initial biomasses for each species as a random value in [0.5,1]
        b_init[node] = np.random.uniform(0.5,1,1)[0]
    niche.write_web_to_file(web,nis,ris,cis,b_init,f"./{dummy_location}/web_{i}") # name changed so not overwritten
    if niche.check_web(web):
        webs0.append(i)
        
print(f"number of recipient webs that pass the check: {len(webs0)}")
valid_webs_file = open(f"./{dummy_location}/valid_webs.txt",'w') # name changed so not overwritten
for i in webs0:
    valid_webs_file.write(str(i) + "\n")
valid_webs_file.close()
        

number of recipient webs that pass the check: 69


In [ ]:
### Run webs for 2000 timesteps to get them to a stable state ###

webs0 = []
valid_webs_file = open(f"./{out_location}/valid_webs.txt",'r')
ct = 0 
for row in valid_webs_file:
    if ct == 0:
        # Only plot time series for the first example 
        webs0.append([f"./{out_location}/web_{int(row)}",2000,True])
    else:
        webs0.append([f"./{out_location}/web_{int(row)}",2000,False])
    ct += 1
valid_webs_file.close()
    
# Parallelize + run (because each simulation takes a while!)
p = Pool(multiprocessing.cpu_count())
p.map(ATN.run_dynamics, webs0)
p.close()
p.join() #wait  

In [3]:
### Count the number of final webs with over 15 species after being run to equilibrium and output a list of those ###
ct = 0
valid_webs_file = open(f"./{out_location}/valid_webs.txt",'r')
fifteen_webs_file = open(f"./{out_location}/over15_webs.txt",'w')
for row in valid_webs_file:
    # Read in the network structure after the 
    web,_,_,_,_ = niche.read_web_from_file(f"./{out_location}/web_{int(row)}_2000")
    final_S = len(web.nodes())
    if final_S >= 15:
        fifteen_webs_file.write(str(row))
        ct+=1
valid_webs_file.close()
fifteen_webs_file.close()
print("count - " + str(ct))

count - 37
